In [57]:
%env JAX_PLATFORM_NAME=cpu

import jaxtyping  # noqa: F401

%load_ext jaxtyping
%jaxtyping.typechecker beartype.beartype

env: JAX_PLATFORM_NAME=cpu
The jaxtyping extension is already loaded. To reload it, use:
  %reload_ext jaxtyping


In [58]:
import jax
import jax.numpy as jnp
import equinox as eqx
import optax

from tqdm.notebook import trange

from jaxtyping import Array, Bool, Float

from chaogatenn.chaogate import ChaoGate
from chaogatenn.maps import LogisticMap
from chaogatenn.utils import grad_norm

In [59]:
# Training data for the AND gate
X = jnp.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=bool)  # Input combinations
AND_Y = jnp.array([0, 0, 0, 1], dtype=bool)  # AND gate output
OR_Y = jnp.array([0, 1, 1, 1], dtype=bool)  # OR gate output
XOR_Y = jnp.array([0, 1, 1, 0], dtype=bool)  # XOR
NAND_Y = jnp.array([1, 1, 1, 0], dtype=bool)  # NAND
XNOR_Y = jnp.array([1, 0, 0, 1], dtype=bool)  # XNOR
Y = XNOR_Y

In [60]:
Map = LogisticMap(a=4.0)
# Map = LorenzMap()
# Map = DuffingMap(
#     alpha=1.0, beta=5.0, delta=0.02, gamma=8.0, omega=0.5, dt=0.01, steps=1000
# )

In [61]:
Map(2)

-8.0

In [62]:
chao_gate = ChaoGate(DELTA=1.0, X0=1.0, X_THRESHOLD=1.0, Map=Map)

In [63]:
[chao_gate(x) for x in X]

[Array(0.26894143, dtype=float32, weak_type=True),
 Array(0.00012339, dtype=float32, weak_type=True),
 Array(0.00012339, dtype=float32, weak_type=True),
 Array(1.3887943e-11, dtype=float32, weak_type=True)]

In [64]:
@eqx.filter_value_and_grad()
def compute_loss(
    chao_gate: ChaoGate, x: Bool[Array, "batch 2"], y: Bool[Array, "batch"]
) -> Float[Array, ""]:  # noqa: F821
    pred = jax.vmap(chao_gate)(x)
    # binary cross entropy
    return -jnp.mean(y * jnp.log(pred + 1e-15) + (1 - y) * jnp.log(1 - pred + 1e-15))

In [65]:
@eqx.filter_jit
def make_step(
    model: ChaoGate,
    x: Bool[Array, "dim 2"],
    y: Bool[Array, "dim"],  # noqa: F821
    optim: optax.GradientTransformation,
    opt_state: optax.OptState,
) -> (Float[Array, "dim"], ChaoGate, optax.OptState):  # type: ignore  # noqa: F821
    loss, grads = compute_loss(model, x, y)
    updates, opt_state = optim.update(grads, opt_state)
    # jax.debug.print(f"{grads, updates}")
    model = eqx.apply_updates(model, updates)
    return loss, model, opt_state

In [66]:
optim = optax.adabelief(3e-4)
opt_state = optim.init(eqx.filter(chao_gate, eqx.is_inexact_array))

In [67]:
epochs = 4_000

for epoch in trange(epochs):
    loss, chao_gate, opt_state = make_step(chao_gate, X, Y, optim, opt_state)  # type: ignore
    _, grads = compute_loss(chao_gate, X, Y)
    grad_norm_value = grad_norm(grads)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss}, Grad Norm: {grad_norm_value}")

  0%|          | 0/4000 [00:00<?, ?it/s]

Epoch 0, Loss: 6.578359127044678, Grad Norm: 11.527771949768066
Epoch 10, Loss: 6.518637180328369, Grad Norm: 11.471450805664062
Epoch 20, Loss: 6.434618949890137, Grad Norm: 11.39284896850586
Epoch 30, Loss: 6.329668998718262, Grad Norm: 11.294770240783691
Epoch 40, Loss: 6.2077131271362305, Grad Norm: 11.180341720581055
Epoch 50, Loss: 6.072001934051514, Grad Norm: 11.052075386047363
Epoch 60, Loss: 5.925110340118408, Grad Norm: 10.911921501159668
Epoch 70, Loss: 5.769141674041748, Grad Norm: 10.761431694030762
Epoch 80, Loss: 5.605870246887207, Grad Norm: 10.601882934570312
Epoch 90, Loss: 5.436831474304199, Grad Norm: 10.434350967407227
Epoch 100, Loss: 5.263375282287598, Grad Norm: 10.25976276397705
Epoch 110, Loss: 5.08670711517334, Grad Norm: 10.078917503356934
Epoch 120, Loss: 4.907907009124756, Grad Norm: 9.892494201660156
Epoch 130, Loss: 4.727951526641846, Grad Norm: 9.701082229614258
Epoch 140, Loss: 4.547721862792969, Grad Norm: 9.505166053771973
Epoch 150, Loss: 4.3680214

In [68]:
print("\nTrained ChaoGate Parameters:")
print(
    f"DELTA: {chao_gate.DELTA}, X0: {chao_gate.X0}, X_THRESHOLD: {chao_gate.X_THRESHOLD}"
)


Trained ChaoGate Parameters:
DELTA: 2.169006984331645e-05, X0: 0.45750245451927185, X_THRESHOLD: 0.992787778377533


In [69]:
[
    (
        bool(x1.item()),
        bool(x2.item()),
        (
            chao_gate.Map(chao_gate.X0 + x1 * chao_gate.DELTA + x2 * chao_gate.DELTA)
            > chao_gate.X_THRESHOLD
        ).item(),
    )
    for x1, x2 in X
]

[(False, False, False),
 (False, True, False),
 (True, False, False),
 (True, True, True)]

In [70]:
pred_ys = jax.vmap(chao_gate)(X)
num_correct = jnp.sum((pred_ys > 0.5) == Y)
final_accuracy = (num_correct / len(X)).item()
print(f"final_accuracy={final_accuracy}")

final_accuracy=0.75
